In [1]:
import cv2
import mediapipe as mp

In [2]:
#Tracker Design:
#variable for drawing interconnected lines
mp_drawing=mp.solutions.drawing_utils
#styling the lines
mp_drawing_styles=mp.solutions.drawing_styles
#helps in tracking the hands in real time
mphands=mp.solutions.hands

In [4]:
# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.6, min_tracking_confidence=0.6)

# Gesture labels
gesture_labels = {0: 'Thumbs Up', 1: 'Thumbs Down', 2: 'Open Palm', 3: 'Closed Fist', 4: 'Peace Sign'}

# Initialize OpenCV VideoCapture
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the image to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process hand detection with MediaPipe
    results = hands.process(rgb_frame)

    if results.multi_hand_landmarks: #If multiple hand landmarks, iterate through each of them
        for hand_landmarks in results.multi_hand_landmarks:
            #Tracker:
            mp_drawing.draw_landmarks(frame,hand_landmarks,mphands.HAND_CONNECTIONS)
            # Calculate hand gesture features
            thumb_tip = hand_landmarks.landmark[4]
            index_tip = hand_landmarks.landmark[8]
            middle_tip = hand_landmarks.landmark[12]
            ring_tip = hand_landmarks.landmark[16]
            pinky_tip = hand_landmarks.landmark[20]
            
# X coordinate -> It indicates the number of pixels from the left edge of the frame to the landmark's position.
# Y coordinate -> It indicates the number of pixels from the top edge of the frame to the landmark's position.
            
            thumb_up = index_tip.y < thumb_tip.y
            thumb_out = index_tip.x < thumb_tip.x
            index_out = middle_tip.y < index_tip.y
            pinky_out = pinky_tip.x < ring_tip.x
            
            # Determine hand side
            hand_side = "Right" if index_tip.x < thumb_tip.x else "Left"

            # Determine hand gesture
            gesture = None
            if not thumb_up and thumb_out:
                gesture = 0  # Thumbs Up
            elif thumb_up and not thumb_out:
                gesture = 1  # Thumbs Down
            elif thumb_up and index_out and pinky_out:
                gesture = 2  # Open Palm
            elif not(thumb_up and index_out and pinky_out):
                gesture = 3  # Closed Fist
            elif not thumb_up and index_out and pinky_out:
                gesture = 4  # Peace Sign
                
                
             # Display hand gesture label
            if gesture is not None:
                text = f"{hand_side} Hand: {gesture_labels[gesture]}"
                # Adjust text size based on frame width
                font_scale = min(frame.shape[1] // 200, 0.6)
                text_thickness = min(frame.shape[1] // 300, 1)
                text_size = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, font_scale, text_thickness)[0]
                text_position = (50, text_size[1] + 20) if hand_side == "Right" else (frame.shape[1] - text_size[0] - 50, text_size[1] + 20)
                cv2.putText(frame, text, text_position,
                            cv2.FONT_HERSHEY_SIMPLEX, font_scale, (0, 0, 255), text_thickness)


    # Display the frame
    cv2.imshow('Hand Gesture Recognition', frame)

    if cv2.waitKey(1) & 0xFF == ord('e'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()
